# BLOCK-T240 - TMA Azimuth Brake Distance

In [ ]:
#%load_ext lab_black
import numpy as np
import os

from lsst.ts.observing import ObservingBlock, ObservingScript

In [ ]:
name = "BLOCK-T3"
program = "BLOCK-T3"
constraints = []
scripts = []

output_folder = "output_blocks"

In [ ]:
def build_configuration_schema(block_number, properties):
    """
    Builds a configuration schema string for a given BLOCK and configurable
    properties.

    Parameters
    ----------
    block_number :
        The BLOCK number to include in the title and description.
    properties : dict
        A dictionary where each key is a property name, and each value is a
        dictionary with keys 'description', 'type', and optionally 'default'.

    Returns
    -------
        A formatted configuration schema string.
    """

    # Define the base schema with the BLOCK number
    configuration_schema = (
        "$schema: http://json-schema.org/draft-07/schema#\n"
        f"title: BLOCK-{block_number} configuration\n"
        f"description: Configuration for BLOCK-{block_number}.\n"
        "type: object\n"
        "properties:\n"
    )

    # Add each property to the schema
    for prop_name, prop_details in properties.items():
        configuration_schema += f"  {prop_name}:\n"
        configuration_schema += f'    description: {prop_details["description"]}\n'
        configuration_schema += f'    type: {prop_details["type"]}\n'
        if "default" in prop_details:
            # Add quotes around the default value if it's a string
            default_value = prop_details["default"]
            if prop_details["type"] == "string":
                default_value = f'"{default_value}"'
            configuration_schema += f"    default: {default_value}\n"

    return configuration_schema

Define the configurable properties that we will use in the configuration schema

In [ ]:
properties = {
    "step_size": {
        "description": "Step size in microns for x and y axes",
        "type": "number",
        "default": "125",
    },
    "max_position": {
        "description": "Absolute maximum position for x and y axes",
        "type": "number",
        "default": "4500",
    }
}

block_number = name.split("-")[-1]
configuration_schema = build_configuration_schema(block_number, properties)
print(configuration_schema)

In [ ]:
# Disable hexapod compensation mode
disable_compensation = ObservingScript(
    name="run_command.py",
    standard=True,
    parameters=dict(
        component="MTHexapod:2",
        cmd="setCompensationMode",
        parameters=dict(mode="off"),
    ),
)

scripts.append(disable_compensation)    

# Zero position block
zero_position = ObservingScript(
    name="run_command.py",
    standard=True,
    parameters=dict(
        component="MTHexapod:2",
        cmd="move",
        parameters=dict(x=0, y=0, z=0, u=0, v=0, w=0),
    )
)

scripts.append(zero_position)

# Warm up z axis
warmup_z = ObservingScript(
    name="maintel/warmup_hexapod.py",
    standard=False,
    parameters=dict(
        hexapod="m2",
        axis="z",
        max_position=5000
    )
)

scripts.append(warmup_z)
scripts.append(zero_position)

# Warm up y axis with xy_step_size and xy_max_position configuration
warmup_y = ObservingScript(
    name="maintel/warmup_hexapod.py",
    standard=False,
    parameters=dict(
        hexapod="m2",
        axis="y",
        step_size="$step_size",
        max_position="$max_position"
    )
)

scripts.append(warmup_y)
scripts.append(zero_position)

# Warm up x axis with xy_step_size and xy_max_position configuration
warmup_x = ObservingScript(
    name="maintel/warmup_hexapod.py",
    standard=False,
    parameters=dict(
        hexapod="m2",
        axis="x",
        step_size="$step_size",
        max_position="$max_position"
    )
)




In [ ]:
block = ObservingBlock(
    name=name,
    program=program,
    configuration_schema=configuration_schema,
    scripts=scripts,
)

In [ ]:
block.model_dump_json(indent=2)

os.makedirs(output_folder, exist_ok=True)
output_path = f"{output_folder}/{name}.json"

with open(output_path, "w") as file:
    file.write(block.model_dump_json(indent=4))